In [2]:
from main import *
YEARS = [2018, 2019, 2021, 2022, 2023, 2024]
SAVE_DIR = "data"

In [ ]:



fetch_lap_data(YEARS, save_dir=SAVE_DIR)
fetch_car_data(YEARS, save_dir=SAVE_DIR)



In [ ]:
fetch_weather_data(YEARS, save_dir="data")
df_weather_all_years = annotate_and_aggregate_weather(YEARS, lap_data_dir="data", weather_dir="data")


In [ ]:
df_aggregate_car = aggregate_car_data(years=YEARS, data_dir="data")

In [ ]:
df_aggregate_car

In [ ]:

# Step 1
df_laps_all = concat_lap_metadata(YEARS, data_dir="data")

# Step 2
df_car_lap = merge_car_with_laps("data/aggregated_car_data_all_years.csv", df_laps_all, output_path="data/car_lap_merged.csv")

# Step 3
final_df = merge_with_weather("data/aggregated_weather_all_years.csv", df_car_lap, output_path="data/merged_lap_car_weather_all_years.csv")


In [ ]:
final_df

In [ ]:
cleanup_intermediate_csvs(directory="data")

In [ ]:
df_weighted = add_simulated_weight_column(
    input_path="data/merged_lap_car_weather_all_years.csv",
    output_path="data/regression_ready_with_weight.csv"
)

In [ ]:
final_df = preprocess_for_regression()


In [ ]:
final_df

In [3]:
df_cleaned = final_cleaning_for_model(input_path='data/regression_final.csv')
df_cleaned

✅ Cleaned regression data saved to data/regression_final_cleaned.csv


,LapNumber,TrackTemp,RPM,Throttle,Brake,DRS,LapTime,IsPersonalBest,Compound,TyreLife,FreshTyre,TrackStatus,Weight,PitDuration
0,1.0,33.75,9312.234987,44.313316,123,0,91.614,0,6.0,20.0,1,1,837.745401,0.0
1,2.0,33.90,9535.940653,46.261128,99,0,80.619,1,1.0,6.0,0,1,836.423392,0.0
2,3.0,33.90,9393.537313,47.238806,93,18,79.812,1,1.0,7.0,0,0,835.101383,0.0
3,4.0,33.90,9425.658683,47.775449,89,22,79.562,1,1.0,8.0,0,0,833.779374,0.0
4,5.0,33.60,9441.933535,48.039275,85,24,79.400,1,1.0,9.0,0,0,832.457366,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8306,72.0,40.50,9011.182410,44.071661,93,0,80.585,0,4.0,2.0,1,0,807.991346,0.0
8307,73.0,40.80,9374.900356,50.697509,90,0,75.233,1,4.0,3.0,1,0,806.597978,0.0
8308,74.0,40.80,8988.793651,44.209524,102,0,80.473,0,4.0,4.0,1,0,805.204610,0.0
8309,75.0,40.60,9442.755474,52.416058,85,0,74.718,1,4.0,5.0,1,0,803.811242,0.0


In [3]:
model, metrics, X = train_random_forest(df_cleaned)


📊 Random Forest Regression Results:
• Mean Squared Error (MSE): 3.48
• Mean Absolute Error (MAE): 0.94
• R² Score: 0.9642


In [ ]:
# Try standard train/test to simulate CV
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = RandomForestRegressor(max_depth=40, n_estimators=300, min_samples_split=2, min_samples_leaf=1)

# 1. Separate features and target
X = df_cleaned.drop(['LapNumber','LapTime', 'FreshTyre', 'IsPersonalBest'], axis=1)
y = df_cleaned['LapTime']

# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2 #, random_state=1
)

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model.fit(X_train, y_train)
    r2 = model.score(X_test, y_test)
    print(f"Fold {fold + 1} R²: {r2:.4f}")

In [ ]:
import matplotlib.pyplot as plt

def plot_feature_importances(model, feature_names):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize=(10, 6))
    plt.title("Feature Importances")
    plt.bar(range(len(importances)), importances[indices])
    plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_feature_importances(model, X.columns)

In [4]:
df_fixed = create_fixed_variables(df_cleaned)
df_fixed

,LapNumber,Brake,DRS,PitDuration,Compound,TyreLife
0,1,124,0,0.0,5,1
1,2,113,0,0.0,5,2
2,3,102,5,0.0,5,3
3,4,98,7,0.0,5,4
4,5,97,7,0.0,5,5
...,...,...,...,...,...,...
73,74,92,3,0.0,6,43
74,75,91,4,0.0,6,44
75,76,92,4,0.0,6,45
76,77,92,4,0.0,6,46


In [5]:
df_random = df_cleaned[['LapNumber','TrackTemp','Weight','TrackStatus','Throttle','RPM']]
df_random

,LapNumber,TrackTemp,Weight,TrackStatus,Throttle,RPM
0,1.0,33.75,837.745401,1,44.313316,9312.234987
1,2.0,33.90,836.423392,1,46.261128,9535.940653
2,3.0,33.90,835.101383,0,47.238806,9393.537313
3,4.0,33.90,833.779374,0,47.775449,9425.658683
4,5.0,33.60,832.457366,0,48.039275,9441.933535
...,...,...,...,...,...,...
8306,72.0,40.50,807.991346,0,44.071661,9011.182410
8307,73.0,40.80,806.597978,0,50.697509,9374.900356
8308,74.0,40.80,805.204610,0,44.209524,8988.793651
8309,75.0,40.60,803.811242,0,52.416058,9442.755474


In [6]:
race_times_df_v10 = monte_carlo_simulation(model, df_fixed, df_random, n_simulations=5000, car_type='v10')
race_times_df_v6 = monte_carlo_simulation(model, df_fixed, df_random, n_simulations=5000, car_type='v6')
race_times_df_v6.to_csv('data/race_times_v6.csv')
race_times_df_v10.to_csv('data/race_times_v10.csv')

In [7]:
race_times_df_v6 = pd.read_csv('data/race_times_v6.csv')
race_times_df_v10 = pd.read_csv('data/race_times_v10.csv')

In [8]:
t_test_hypothesis1(race_times_df_v10, race_times_df_v6)

--- T-Test Results ---
T-statistic = -37.8703
P-value = 0.0000000000
✅ **Reject Null Hypothesis**: V10 hybrid engines yield significantly faster race times than V6 (p < 0.05).

Mean Race Time (V10): 6334.28 seconds
Mean Race Time (V6): 6386.21 seconds
Difference: 51.93 seconds (V6 - V10)
